In [ ]:
from pathlib import Path
import torch
import json
import pandas as pd
import tqdm

from au2v.config import ModelConfig, TrainerConfig
from au2v.dataset_manager import load_dataset_manager
from au2v.trainer import PyTorchTrainer
from au2v.model import load_model, PyTorchModel

In [ ]:
def calc_accuracy(
    model: PyTorchModel,
    num_item: int,
    test_dataset: list[tuple[int, list[int], list[int]]],
    top_k: list[int],
) -> dict[str, float]:
    hit_counts = {k: 0 for k in top_k}
    for seq_index, context_items, target_indices in tqdm.tqdm(test_dataset):
        rec_list = model.output_rec_lists(
            seq_index=torch.LongTensor([seq_index]),
            item_indices=torch.LongTensor([context_items]),
            cand_item_indices=torch.arange(num_item),
            k=max(top_k),
        )
        for k in top_k:
            hit_counts[k] += len(set(target_indices) & set(rec_list[0][:k]))

    total_rec = len(test_dataset)
    results = {
        f"Accuracy@{k}": hit_count / total_rec / k
        for k, hit_count in hit_counts.items()
    }
    return results

In [ ]:
run_configs = [
    # {
    #     "name": "User2Vec (d=16)",
    #     "model_name": "doc2vec",
    #     "d_model": 16,
    #     "epochs": 3,
    #     "use_weight_tying": True,
    #     "use_attention": True,
    #     "use_meta": True,
    # },
    # {
    #     "name": "User2Vec (d=32)",
    #     "model_name": "doc2vec",
    #     "d_model": 32,
    #     "epochs": 3,
    #     "use_weight_tying": True,
    #     "use_attention": True,
    #     "use_meta": True,
    # },
    # {
    #     "name": "User2Vec (d=64)",
    #     "model_name": "doc2vec",
    #     "d_model": 64,
    #     "epochs": 10,
    #     "use_weight_tying": True,
    #     "use_attention": True,
    #     "use_meta": True,
    # },
    {
        "name": "AU2V (d=16)",
        "model_name": "attentive",
        "d_model": 16,
        "epochs": 5,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AU2V (d=32)",
        "model_name": "attentive",
        "d_model": 32,
        "epochs": 5,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AU2V (d=64)",
        "model_name": "attentive",
        "d_model": 64,
        "epochs": 5,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    # {
    #     "name": "AU2V (wo weight-tying)",
    #     "model_name": "attentive",
    #     "d_model": 64,
    #     "epochs": 3,
    #     "use_weight_tying": False,
    #     "use_attention": True,
    #     "use_meta": True,
    # },
    # {
    #     "name": "AU2V (wo attention)",
    #     "model_name": "attentive",
    #     "d_model": 64,
    #     "epochs": 3,
    #     "use_weight_tying": True,
    #     "use_attention": False,
    #     "use_meta": True,
    # },
    # {
    #     "name": "AU2V (wo meta)",
    #     "model_name": "attentive",
    #     "d_model": 64,
    #     "epochs": 3,
    #     "use_weight_tying": True,
    #     "use_attention": True,
    #     "use_meta": False,
    # },
]

In [ ]:
results = {}

for run_config in run_configs:
    results[run_config["name"]] = {"accuracy": []}
    model_config = ModelConfig(
        weight_decay=1e-8,
        max_embedding_norm=5,
        d_model=run_config["d_model"],
        lr=5e-5,
        use_attention=run_config["use_attention"],
        use_meta=run_config["use_meta"],
        use_weight_tying=run_config["use_weight_tying"],
    )
    trainer_config = TrainerConfig(
        dataset_name="movielens",
        model_name=run_config["model_name"],
        load_dataset=False,
        save_dataset=False,
        load_model=False,
        ignore_saved_model=True,
        epochs=run_config["epochs"],
    )

    print(model_config)
    print(trainer_config)

    dataset_manager = load_dataset_manager(
        dataset_name=trainer_config.dataset_name,
        dataset_dir=trainer_config.dataset_dir,
        data_dir="../data/",
        load_dataset=trainer_config.load_dataset,
        save_dataset=trainer_config.save_dataset,
        window_size=model_config.window_size,
    )
    print(
        "train:",
        len(dataset_manager.train_dataset),
        "valid:",
        len(dataset_manager.valid_dataset),
    )
    model = load_model(
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer = PyTorchTrainer(
        model=model,
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )

    def on_epoch_end(epoch: int):
        result = calc_accuracy(
            model=model,
            num_item=dataset_manager.num_item,
            test_dataset=dataset_manager.test_datasets["test"],
            top_k=[10, 20, 30, 40, 50],
        )
        print(epoch, result)
        results[run_config["name"]]["accuracy"].append(result)
        torch.save(model, f"cache/model/movielens-2/{run_config['name']}-{epoch}.pt")

    losses = trainer.fit(on_epoch_end=on_epoch_end)
    results[run_config["name"]]["loss"] = losses

    with open("result.json", "w") as f:
        json.dump(results, f)

In [ ]:
top_k = list(range(10, 51, 10))
data = {}
for method, result in results.items():
    data[method] = []
    for k in top_k:
        a = max(map(lambda r: r[f"Accuracy@{k}"], result["accuracy"]))
        data[method].append(a)

In [ ]:
df = pd.DataFrame(data).T
df.columns = [f"Accuracy@{k}" for k in top_k]
df